# Data Visualisation and Analytics

## Assignment 2: Using an API and Visualising Your Data

In [1]:
# Imports
import requests
import pandas as pd
import openpyxl
import altair as alt

In [2]:
# Default filepaths
DATA_XLSX = "data.xlsx"

### Download and Read Dataset

In [ ]:
# Dataset: Gender Datenreport Berlin 2022 - Schul- und Berufsbildung
# Source: https://daten.berlin.de/datensaetze/gender-datenreport-berlin-2022-schul-und-berufsbildung-1505506
# Last accessed 25.11.2024

url = "https://www.berlin.de/sen/frauen/gleichstellung/gender-daten/daten/2022_2_1_schule-berufsbildung_20241028_opendata.xlsx"

In [4]:
# Download file
response = requests.get(url)

if response.status_code == 200:
    with open(DATA_XLSX, "wb") as f:
        f.write(response.content)
    print(f"File downloaded and saved as {DATA_XLSX}")
else:
    print(f"Fail to download file. HTTP Status Code: {response.status_code}")

File downloaded and saved as data.xlsx


In [5]:
# Function to get specific dataset in the excel file
def get_data(sheet_name: str, xls_file=DATA_XLSX):
    try:
        df = pd.read_excel(DATA_XLSX, sheet_name=sheet_name, skiprows=1)
        print(df.head())
    except ValueError as e:
        print(f"Error: {e}.")

        print("Available sheets are:")
        with pd.ExcelFile(DATA_XLSX) as xls:
            for i, sheet_name in enumerate(xls.sheet_names):
                print(f"{i:2d}. {sheet_name}")
    
    return df

### Select relavent dataset of interest (1)
- 4 Höchster beruflicher Bildungsabschluss der Berlinerinnen und Berliner im Alter von 25 bis unter 65 Jahren von 2010 bis 2022(2) (Quelle: Mikrozensus)  
    (Translation) Highest vocational qualification of Berliners aged 25 to under 65 from 2010 to 2022

**Analysis Question 1:**  
From 2010 to 2022, is there a general increase in the numbers of Berliners with a Bachelor qualification?

In [6]:
df = get_data("Tab4")

   Jahr(3) Geschlecht  Lehre/Berufs-ausbildung  Fachschul-abschluss  Bachelor  \
0     2010     Frauen                   415800                75300     17100   
1     2010     Männer                   439700                51800     16800   
2     2011     Frauen                   399100                84900     19400   
3     2011     Männer                   416900                56100     18000   
4     2012     Frauen                   400600                79100     19700   

   Master  Diplom  Promotion  Ohne beruflichen Bildungsabschluss  
0   10300  224300      17400                              205200  
1   10900  225700      27300                              218200  
2   16700  221000      15100                              184700  
3   11900  221000      25900                              196600  
4   17300  241700      16300                              184300  


In [ ]:
chart = alt.Chart(df).mark_bar().encode(
    x="Jahr(3):O",              # Year on x-axis (Ordinal)
    y="Bachelor:Q",             # Number of Bachelor degrees on y-axis (Quantitative)
    color="Geschlecht:N",       # Color by gender (Nominal)
    tooltip=['Jahr(3)', 'Geschlecht', 'Bachelor'],
).properties(
    title="Number of Berliners with Bachelor qualification"
).facet(
    column="Geschlecht:N"
)
chart.show()

alt.FacetChart(...)

**Analysis:** There is a general increase in the numbers of Berliners with a Bachelor qualification for both males and females. 
- In 2019, there is a higher number of females with Bachelor than in 2020.

### Select relavent dataset of interest (2)
- 7 Anzahl von Schülerinnen und Schülern an allgemeinbildenden Schulen in Berlin 2000/01 bis 2022/23 (17)  (Quelle: Schulstatistik der Senatsverwaltung für Bildung, Jugend und Familie)  
    (Translation) Number of pupils at general education schools in Berlin 2000/01 to 2022/23

**Analysis Question 2:**  
Does the trend of the percentage of foreign students in a school year align with the trend of the percentage of students with non-German native language?

In [8]:
df = get_data("Tab7")

  Schuljahr(4)  Deutsche Schülerinnen  Deutsche Schüler  \
0      2000/01                 159442            161811   
1      2001/02                 153756            155877   
2      2002/03                 149103            150739   
3      2003/04                 145271            146806   
4      2004/05                 141729            143212   

   Ausländische Schülerinnen  Ausländische Schüler  \
0                      27164                 28965   
1                      27029                 28951   
2                      27189                 29084   
3                      27195                 28876   
4                      27047                 28670   

   NDH (nichtdeutsche Herkunftssprache) (5)  
0                                       NaN  
1                                       NaN  
2                                       NaN  
3                                       NaN  
4                                       NaN  


In [9]:
# Step 1: Filter out rows where NDH is NaN
df = df.loc[df['NDH (nichtdeutsche Herkunftssprache) (5)'].notna()]

# Step 2: Calculate the total number of students for each year
df['Total Students'] = df[['Deutsche Schülerinnen', 'Deutsche Schüler', 
                            'Ausländische Schülerinnen', 'Ausländische Schüler']].sum(axis=1)

# Step 3: Calculate the percentage of German and foreign students (girls + boys)
df['Deutsche Studenten'] = (df['Deutsche Schülerinnen'] + df['Deutsche Schüler']) / df['Total Students'] * 100
df['Ausländische Studenten'] = (df['Ausländische Schülerinnen'] + df['Ausländische Schüler']) / df['Total Students'] * 100

# Step 4: Calculate the percentage of NDH students relative to the total number of students
df['NDH Percentage'] = df['NDH (nichtdeutsche Herkunftssprache) (5)'] / df['Total Students'] * 100

df.head()

,Schuljahr(4),Deutsche Schülerinnen,Deutsche Schüler,Ausländische Schülerinnen,Ausländische Schüler,NDH (nichtdeutsche Herkunftssprache) (5),Total Students,Deutsche Studenten,Ausländische Studenten,NDH Percentage
5,2005/06,141315,143959,27379,28975,87857.0,341628,83.504280,16.495720,25.717154
6,2006/07,139077,141588,26363,27851,90444.0,334879,83.810869,16.189131,27.007964
7,2007/08,137164,139530,25164,26522,93630.0,328380,84.260308,15.739692,28.512699
8,2008/09,135757,138435,23878,25150,95316.0,323220,84.831384,15.168616,29.489512
9,2009/10,135160,138761,23052,23897,98816.0,320870,85.368218,14.631782,30.796273


In [10]:
chart1 = alt.Chart(df).mark_bar().encode(
    x='Schuljahr(4):O',
    y='Ausländische Studenten:Q',
    color=alt.value('steelblue'),
    tooltip=['Schuljahr(4)', 'Ausländische Studenten']
).properties(
    title='Percentage of Foreign Students in a School Year'
)
chart1.show()


alt.Chart(...)

In [12]:
chart2 = alt.Chart(df).mark_bar().encode(
    x='Schuljahr(4):O',
    y='NDH Percentage:Q',
    color=alt.value('orange'),
    tooltip=['Schuljahr(4)', 'NDH Percentage']
).properties(
    title='Percentage of Students with non-German as native language in a school year'
)
chart2.show()

alt.Chart(...)

**Analysis:** The trend of the percentage of foreign students in a school year does not align with the trend of the percentage of students with a non-German native language.  
- The percentage of foreign students show a bimodial distribution from School Year 2005/06 to 2022/23.
- The percentage of students with non-German native language shows an increasing trend from from School Year 2005/06 to 2022/23.